In [1]:
import pybullet as p
import time
import numpy as np
import pybullet_data
from robot import Robot
from utils import *
from kcontroller import WQPController
from utils import Tmat, quaternionMult


In [2]:
#función para facilitar tareas
def task(qdes):
        robot.update_config(qdes)
        dt = 0.005
        weights = [10.0, 10.0, 10.0, 10.0]
        lambdas = [0.5, 0.5, 0.5, 0.5]
        solver = WQPController(weights, lambdas, dt)
        pfr_des = robot.fkine_fright()[0:3,3]
        pfl_des = robot.fkine_fleft()[0:3,3]
        prr_des = robot.fkine_rright()[0:3,3]
        prl_des = robot.fkine_rleft()[0:3,3]

        N = 200
        num_joints = 12
        q = np.copy(qdes)
        for i in range(N):
    # Valores articulares deseados
     # Errors
            efr = robot.error_position_fright(pfr_des)
            efl = robot.error_position_fleft(pfl_des)
            err = robot.error_position_rright(prr_des)
            erl = robot.error_position_rleft(prl_des)
    # Task Jacobians
            Jfr = robot.taskj_position_fright()
            Jfl = robot.taskj_position_fleft()
            Jrr = robot.taskj_position_rright()
            Jrl = robot.taskj_position_rleft()
    
    
        # Get the joint velocity
            dq = solver.get_dq(q, efr, Jfr, efl, Jfl, err, Jrr, erl, Jrl)
    
    # Integrate rotation
            w = np.dot(Tmat(q), dq[3:7])
            dth = np.linalg.norm(w)
            if abs(dth)>1e-9:
                u = w/dth
                dQ = np.array([np.cos(dth*dt/2.0), u[0]*np.sin(dth*dt/2.0),
                       u[1]*np.sin(dth*dt/2.0), u[2]*np.sin(dth*dt/2.0)])
                Q = quaternionMult(dQ, q[3:7])
                q[3:7] = Q
    # Integrate position and joint configuration
            q[0:3] = q[0:3] + dt*dq[0:3]
            q[7:]  = q[7:] + dt*dq[7:]

        return q

In [3]:
# Conexión al simulador físico
physicsClient = p.connect(p.GUI)
# Ruta a los archivos por defecto de PyBullet 
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Resetear el simulador 
p.resetSimulation()
# Gravedad del simulador
p.setGravity(0,0,-9.81)
# Punto de vista inicial
p.resetDebugVisualizerCamera(cameraDistance=1.9, cameraYaw=130, cameraPitch=-20, 
                             cameraTargetPosition=[0,0,0])
# Añadir un plano
plano = p.loadURDF("plane.urdf")
# Añadir el robot Laikago
start_pos = [-0.0, 0, 0.55]
start_orient = p.getQuaternionFromEuler([np.pi/2, 0, np.pi/2])
laikago = p.loadURDF("laikago/laikago.urdf", start_pos, start_orient)
robot = Robot()
# Índices de las articulaciones actuadas
id_joints = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
len(id_joints)

12

In [4]:
# Obtener la posición y orientación actuales de la base del robot
robot_pos, robot_or = p.getBasePositionAndOrientation(laikago)

print("Posición del robot (x,y,z):", np.round(robot_pos,3))
print("Orientación del robot (ex,ey,ez,w):", np.round(robot_or,3))

Posición del robot (x,y,z): [0.044 0.    0.58 ]
Orientación del robot (ex,ey,ez,w): [0.5 0.5 0.5 0.5]


In [5]:
# Se añade una pequeña pelotita para verificar visualmente la cinemática
ball1 = p.loadURDF("sphere_small.urdf", [1,2,2])  
# Se añade una pequeña pelotita para verificar visualmente la cinemática
ball2 = p.loadURDF("sphere_small.urdf", [2,2,2])
# Se añade una pequeña pelotita para verificar visualmente la cinemática
ball3 = p.loadURDF("sphere_small.urdf", [3,2,2])
# Se añade una pequeña pelotita para verificar visualmente la cinemática
ball4 = p.loadURDF("sphere_small.urdf", [4,2,2])

In [6]:
#CONTROL CINEMATICO DEL MODELO
# Configuración inicial del robot (impuesta arbitrariamente)
q0 = np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
               0., 0., -0.5,#derecha delantera
               0., 0., -0.5,#izquierda delantera
               0., 0., -0.5, #deracha trasera
               0., 0., -0.5])#izquierda trasera

# q0=np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
#               0.2, 0.3, 0.5,#derecha delantera
#               0.2, 0.3, 0.,#izquierda delantera
#               0.2, 0.3, 0., #deracha trasera
#               0.2, 0.3, 0.])#izquierda trasera
##calculo de la CINEMATICA DIRECTA

robot = Robot()
robot.update_config(q0)
# Configuracion Inicial
q_pb = q0[7:19]
for i, joint in enumerate(id_joints): 
    p.resetJointState(laikago, joint, q_pb[i])

xfr = robot.fkine_fright()
print(xfr[0:3,3])
# Front left foot
xfl = robot.fkine_fleft()
print(xfl[0:3,3])
# Rear right foot
xrr = robot.fkine_rright()
print(xrr[0:3,3])
# Rear left foot
xrl = robot.fkine_rleft()
print(xrl[0:3,3])


# p.resetBasePositionAndOrientation(ball1, xfr[0:3,3], [0,0,0,1])
# p.resetBasePositionAndOrientation(ball1, xfr[0:3,3], [0,0,0,1])
# p.resetBasePositionAndOrientation(ball2, xfl[0:3,3], [0,0,0,1])
# p.resetBasePositionAndOrientation(ball3, xrr[0:3,3], [0,0,0,1])
# p.resetBasePositionAndOrientation(ball4, xrl[0:3,3], [0,0,0,1])


[ 0.18981542 -0.12        0.07399338]
[0.19162974 0.12       0.04411348]
[-0.24518458 -0.12        0.07399338]
[-0.24337026  0.12        0.04411348]


In [9]:
qdes1=np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
               0.0, -0.8, -0.5,#derecha delantera
               0., 0., -0.5,#izquierda delantera
               0., 0., -0.5, #deracha trasera
               0.0, -0.8, -0.5])#izquierda trasera

qdes2=np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
               0., 0., -0.5,#derecha delantera
               0.0, -0.8, -0.5,#izquierda delantera
               0.0, -0.8, -0.5, #deracha trasera
               0., 0., -0.5])#izquierda trasera

q1=task(qdes1)
q2=task(qdes2)
#lai=1
for lai in range(8):
    if(lai % 2 != 0):
        robot.update_config(q1)
        for i, joint in enumerate(id_joints): 
            p.resetJointState(laikago, joint, q1[7+i])
        time.sleep(0.5)
    else:    
        robot.update_config(q2)
        for i, joint in enumerate(id_joints): 
            p.resetJointState(laikago, joint, q2[7+i])
        time.sleep(0.5)
#robot.update_config(q1)
#for i, joint in enumerate(id_joints): 
#    p.resetJointState(laikago, joint, q1[7+i])
    
# robot.update_config(q1)
# xfr = robot.fkine_fright()
# # Front left foot
# xfl = robot.fkine_fleft()
# # Rear right foot
# xrr = robot.fkine_rright()
# # Rear left foot
# xrl = robot.fkine_rleft()

# p.resetBasePositionAndOrientation(ball1, xfr[0:3,3], [0,0,0,1])
# p.resetBasePositionAndOrientation(ball2, xfl[0:3,3], [0,0,0,1])
# p.resetBasePositionAndOrientation(ball3, xrr[0:3,3], [0,0,0,1])
# p.resetBasePositionAndOrientation(ball4, xrl[0:3,3], [0,0,0,1])

In [88]:
#Control qp para posición, si sale soy chalon
#qdes=np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
#               0.2, 0.3, 0.,#derecha delantera
#               0.1, -0.1, 0.,#izquierda delantera
#               0.2, 0.3, 0., #deracha trasera
#               0.1, -0.1, 0.])#izquierda trasera

qdes=np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
               0.0, -0.8, -0.9,#derecha delantera
               0.0, 0., -0.5,#izquierda delantera
               0.0, 0., -0.5, #deracha trasera
               0., 0., -0.5])#izquierda trasera
robot.update_config(qdes)
dt = 0.005
weights = [1.0, 10.0, 10.0, 10.0]
#lambdas = [0.01, 0.01, 0.01, 0.01]
#lambdas = [0.1, 0.1, 0.1, 0.1]
lambdas = [0.5, 0.5, 0.5, 0.5]
#lambdas = [1.0, 1.0, 1.0, 1.0]
#lambdas = [10.0, 10.0, 10.0, 10.0]
solver = WQPController(weights, lambdas, dt)
pfr_des = robot.fkine_fright()[0:3,3]
pfl_des = robot.fkine_fleft()[0:3,3]
prr_des = robot.fkine_rright()[0:3,3]
prl_des = robot.fkine_rleft()[0:3,3]

# Change initial configuration
#pfr_des[2] = 0.15
#pfr_des[1] += 0.05


#q = np.array([state[0] for state in p.getJointStates(laikago, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])])

N = 200
#delta = (qdes[7:19]-q)/N

#Movimento del laikago
#dt = 0.005
num_joints = 12
q = np.copy(qdes)
for i in range(N):
    # Valores articulares deseados
     # Errors
    efr = robot.error_position_fright(pfr_des)
    efl = robot.error_position_fleft(pfl_des)
    err = robot.error_position_rright(prr_des)
    erl = robot.error_position_rleft(prl_des)
    # Task Jacobians
    Jfr = robot.taskj_position_fright()
    Jfl = robot.taskj_position_fleft()
    Jrr = robot.taskj_position_rright()
    Jrl = robot.taskj_position_rleft()
    
    
        # Get the joint velocity
    dq = solver.get_dq(q, efr, Jfr, efl, Jfl, err, Jrr, erl, Jrl)
    
    # Integrate rotation
    w = np.dot(Tmat(q), dq[3:7])
    dth = np.linalg.norm(w)
    if abs(dth)>1e-9:
        u = w/dth
        dQ = np.array([np.cos(dth*dt/2.0), u[0]*np.sin(dth*dt/2.0),
                       u[1]*np.sin(dth*dt/2.0), u[2]*np.sin(dth*dt/2.0)])
        Q = quaternionMult(dQ, q[3:7])
        q[3:7] = Q
    # Integrate position and joint configuration
    q[0:3] = q[0:3] + dt*dq[0:3]
    q[7:]  = q[7:] + dt*dq[7:]
    # Envío directo de valores articulares deseados al robot (sin simulación dinámica)
#    for i in range(num_joints): p.resetJointState(laikago, i, q[i])
#    #print(qtarget)     
#    time.sleep(dt)
robot = Robot()
robot.update_config(q0)
for i, joint in enumerate(id_joints): 
    p.resetJointState(laikago, joint, q[7+i])
#    #print(qtarget)     
#    time.sleep(dt)

print(robot.error_position_fright(pfr_des))
print(robot.error_position_fleft(pfl_des))
print(robot.error_position_rright(prr_des))
print(robot.error_position_rleft(prl_des))
print(q)
robot.update_config(q)
#print(q2)
xfr = robot.fkine_fright()
print(xfr[0:3,3])
# Front left foot
xfl = robot.fkine_fleft()
print(xfl[0:3,3])
# Rear right foot
xrr = robot.fkine_rright()
print(xrr[0:3,3])
# Rear left foot
xrl = robot.fkine_rleft()
print(xrl[0:3,3])

p.resetBasePositionAndOrientation(ball1, xfr[0:3,3], [0,0,0,1])
p.resetBasePositionAndOrientation(ball2, xfl[0:3,3], [0,0,0,1])
p.resetBasePositionAndOrientation(ball3, xrr[0:3,3], [0,0,0,1])
p.resetBasePositionAndOrientation(ball4, xrl[0:3,3], [0,0,0,1])

#robot = Robot()
#robot.update_config(q)

[0.55140628 0.         0.14021837]
[ 0.12888391  0.         -0.02345295]
[ 0.12812244  0.         -0.02730833]
[ 0.12888391  0.         -0.02345295]
[ 0.044  0.     0.58   1.     0.     0.     0.     0.    -0.8   -0.9
  0.     0.    -0.5    0.     0.    -0.5    0.     0.    -0.5  ]
[ 0.61309927 -0.12        0.24152008]
[0.19162974 0.12       0.04411348]
[-0.24518458 -0.12        0.07399338]
[-0.24337026  0.12        0.04411348]


In [58]:
# Hacer que el robot "baje" hasta el suelo (simulación dinámica)
for i in range(200):
    p.stepSimulation()
    time.sleep(0.005)

In [59]:
# Activar simulación dinámica
p.setRealTimeSimulation(1)

# Velocidad deseada (final) para cada articulación
dqdes = 12*[0]
# Configuración articular deseada para cada articulación
qdes = 4*[0, 0, 1.5]
# Valores articulares actuales
q = np.array([state[0] for state in p.getJointStates(laikago, id_joints)])

# Incremento en valores articulares
N = 200
delta = (qdes-q)/N

# Movimiento del robot (simulación dinámica)
dt = 0.001
for i in range(N):
    qtarget = q + delta*i
    p.setJointMotorControlArray(laikago, id_joints, p.POSITION_CONTROL,
                                qtarget, dqdes, positionGains = 12*[0.03], 
                                velocityGains = 12*[1])
    time.sleep(dt)

# Esperar 2 segundos más
time.sleep(2)
# Detener la simulación dinámica
p.setRealTimeSimulation(0)
print(qtarget)

TypeError: an integer is required (got type Robot)

In [34]:
p.disconnect()

In [94]:
# Activar simulación dinámica
p.setRealTimeSimulation(1)
njoints=12
# Velocidad deseada (final) para cada articulación
dqdes = 12*[0]
# Configuración articular deseada para cada articulación
#q=np.coy(q0)
robot.update_config(q0)
qdes=np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
               0.0, -0.8, -0.9,#derecha delantera
               0.0, 0., -0.5,#izquierda delantera
               0.2, 0., -0.5, #deracha trasera
               0.2, 0., -0.5])#izquierda trasera
dt = 0.005
weights = [1.0, 10.0, 10.0, 10.0]
#lambdas = [0.01, 0.01, 0.01, 0.01]
#lambdas = [0.1, 0.1, 0.1, 0.1]
lambdas = [0.5, 0.5, 0.5, 0.5]
#lambdas = [1.0, 1.0, 1.0, 1.0]
#lambdas = [10.0, 10.0, 10.0, 10.0]
solver = WQPController(weights, lambdas, dt)
pfr_des = robot.fkine_fright()[0:3,3]
pfl_des = robot.fkine_fleft()[0:3,3]
prr_des = robot.fkine_rright()[0:3,3]
prl_des = robot.fkine_rleft()[0:3,3]

# Change initial configuration
#pfr_des[2] = 0.15
#pfr_des[1] += 0.05


#q = np.array([state[0] for state in p.getJointStates(laikago, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])])

N = 200
#delta = (qdes[7:19]-q)/N

#Movimento del laikago
#dt = 0.005
num_joints = 12
q = np.copy(qdes)
robot.update_config(q)
for i in range(N):
    # Valores articulares deseados
     # Errors
    efr = robot.error_position_fright(pfr_des)
    efl = robot.error_position_fleft(pfl_des)
    err = robot.error_position_rright(prr_des)
    erl = robot.error_position_rleft(prl_des)
    # Task Jacobians
    Jfr = robot.taskj_position_fright()
    Jfl = robot.taskj_position_fleft()
    Jrr = robot.taskj_position_rright()
    Jrl = robot.taskj_position_rleft()
    
    
        # Get the joint velocity
    dq = solver.get_dq(q, efr, Jfr, efl, Jfl, err, Jrr, erl, Jrl)
    
    # Integrate rotation
    w = np.dot(Tmat(q), dq[3:7])
    dth = np.linalg.norm(w)
    if abs(dth)>1e-9:
        u = w/dth
        dQ = np.array([np.cos(dth*dt/2.0), u[0]*np.sin(dth*dt/2.0),
                       u[1]*np.sin(dth*dt/2.0), u[2]*np.sin(dth*dt/2.0)])
        Q = quaternionMult(dQ, q[3:7])
        q[3:7] = Q
    # Integrate position and joint configuration
    q[0:3] = q[0:3] + dt*dq[0:3]
    q[7:]  = q[7:] + dt*dq[7:]
    p.setJointMotorControlArray(laikago, id_joints, p.POSITION_CONTROL,
                                q[7:19], dqdes, positionGains = njoints*[0.03], 
                                velocityGains = njoints*[1])
    time.sleep(dt)
    
# Esperar 2 segundos más
time.sleep(2)
# Detener la simulación dinámica
p.setRealTimeSimulation(0)


TypeError: 'NoneType' object is not subscriptable

In [72]:
# Activar simulación dinámica
p.setRealTimeSimulation(1)
njoints=12
# Velocidad deseada (final) para cada articulación
dqdes = 12*[0]
# Configuración articular deseada para cada articulación
#q=np.coy(q0)
robot.update_config(q0)
qdes=np.array([0.044, 0.0, 0.58, 1, 0.0, 0.0, 0.0,
               0.0, -0.8, -0.9,#derecha delantera
               0.0, 0., -0.5,#izquierda delantera
               0.2, 0., -0.5, #deracha trasera
               0.2, 0., -0.5])#izquierda trasera
dt = 0.005
weights = [1.0, 10.0, 10.0, 10.0]
#lambdas = [0.01, 0.01, 0.01, 0.01]
#lambdas = [0.1, 0.1, 0.1, 0.1]
lambdas = [0.5, 0.5, 0.5, 0.5]
#lambdas = [1.0, 1.0, 1.0, 1.0]
#lambdas = [10.0, 10.0, 10.0, 10.0]
solver = WQPController(weights, lambdas, dt)
pfr_des = robot.fkine_fright()[0:3,3]
pfl_des = robot.fkine_fleft()[0:3,3]
prr_des = robot.fkine_rright()[0:3,3]
prl_des = robot.fkine_rleft()[0:3,3]

# Change initial configuration
#pfr_des[2] = 0.15
#pfr_des[1] += 0.05


#q = np.array([state[0] for state in p.getJointStates(laikago, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])])

N = 200
#delta = (qdes[7:19]-q)/N

#Movimento del laikago
#dt = 0.005
num_joints = 12
q = np.copy(qdes)
robot.update_config(q)
for i in range(N):
    # Valores articulares deseados
     # Errors
    efr = robot.error_position_fright(pfr_des)
    efl = robot.error_position_fleft(pfl_des)
    err = robot.error_position_rright(prr_des)
    erl = robot.error_position_rleft(prl_des)
    # Task Jacobians
    Jfr = robot.taskj_position_fright()
    Jfl = robot.taskj_position_fleft()
    Jrr = robot.taskj_position_rright()
    Jrl = robot.taskj_position_rleft()
    
    
        # Get the joint velocity
    dq = solver.get_dq(q, efr, Jfr, efl, Jfl, err, Jrr, erl, Jrl)
    
    # Integrate rotation
    w = np.dot(Tmat(q), dq[3:7])
    dth = np.linalg.norm(w)
    if abs(dth)>1e-9:
        u = w/dth
        dQ = np.array([np.cos(dth*dt/2.0), u[0]*np.sin(dth*dt/2.0),
                       u[1]*np.sin(dth*dt/2.0), u[2]*np.sin(dth*dt/2.0)])
        Q = quaternionMult(dQ, q[3:7])
        q[3:7] = Q
    # Integrate position and joint configuration
    q[0:3] = q[0:3] + dt*dq[0:3]
    q[7:]  = q[7:] + dt*dq[7:]
    p.setJointMotorControlArray(laikago, id_joints, p.POSITION_CONTROL,
                                q[7:19], dqdes, positionGains = njoints*[0.03], 
                                velocityGains = njoints*[1])
    time.sleep(dt)
    
# Esperar 2 segundos más
time.sleep(2)
# Detener la simulación dinámica
p.setRealTimeSimulation(0)


TypeError: 'NoneType' object is not subscriptable

In [11]:
p.disconnect()